In [1]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import datetime

In [2]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [3]:
dfs = pd.read_csv("inputs/dfs.csv")
fd = pd.read_csv("inputs/fanduel.csv")
dk = pd.read_csv("inputs/draftkings.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
fd = fd.drop("Unnamed: 0", axis=1)
dk = dk.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [4]:
ts = datetime.now()
ts = ts.strftime("%b %d, %Y %I:%M %p")
ts = "Last Updated: " + ts

In [5]:
def highlight_dfs(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*19
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*19
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*19
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*19
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*19
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*19
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*19
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*19
    else: return ['background-color: white']*19

In [6]:
def highlight_fd(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*14
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*14
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*14
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*14
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*14
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*14
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*14
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*14
    else: return ['background-color: white']*14

In [7]:
def emojize(df):
    for i in range(len(df)):
        if df['Hot'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n🔥')
        if df['Hot'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Hot'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
        if df['Cold'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
        if df['Cold'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
    return df

In [8]:
def gen_page(df, out, coloring):
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    images = []
    for i in df["Name"]:
        code = "https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg"
        if img_dict.get(i) != None:
            code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(i)[0] + ".jpg"
        images.append(code)
    df["Picture"] = images
    
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    pre = (
        "<head>\n<script src=\"https://kryogenix.org/code/browser/sorttable/sorttable.js\"></script>\n"
        "<link rel=\"stylesheet\" href=\"df_styles.css\">\n</head>\n"
    )
    pre = pre + ts + "\n"
    pre += (
        "\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n"
        "<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<input type=\"text\" id=\"position\" onkeyup=\"posSearch()\" placeholder=\"Search for position...\">\n"
        "</div>\n\n"
    )
    
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    # render dataframe as html
    if coloring is not None:
        images = ['<img src="' + image + '" width="60">' for image in images]
        df["Picture"] = images
        df = emojize(df)
        body = df.style.apply(coloring, axis=1).hide_columns(['Hot', 'Cold']).render()
    else:
        html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
        body = html.data
    body = body.replace("dataframe", "sortable", 1)
    body = body.replace("class=\"sortable\"", "id=\"tbl\" class=\"sortable\"", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)
    return df

In [11]:
gen_page(dfs, 'dfs.html', highlight_dfs).head(20)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

,Name,Picture,Position,Game,Team,Projected Fanduel Points,Projected Fanduel Value,Fanduel Salary,Value above Fanduel Value,Projected Draftkings Points,Projected Draftkings Value,FPPG (Fanduel),10 Game Average (Fanduel),3 Game Average (Fanduel),Opponent Defensive Rank vs Position,Injury Indicator,Injury Details,Hot,Cold
1,LeBron James,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/jamesle01.jpg"" width=""60"">",SF,LAC@LAL,LAL,73.85,14771,11400,$3371,78.12,14648,52.12,56.57,58.53,7.0,,,0.000000,0.000000
2,Anthony Davis,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/davisan02.jpg"" width=""60"">",PF,LAC@LAL,LAL,55.53,11105,10400,$705,55.11,10333,51.92,52.29,44.27,14.0,GTD,Eye,0.000000,0.000000
3,Kawhi Leonard,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/leonaka01.jpg"" width=""60"">",SF,LAC@LAL,LAC,50.96,10192,9500,$692,52.40,9825,47.63,43.88,39.80,7.0,,,0.000000,0.000000
4,Jrue Holiday,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/holidjr01.jpg"" width=""60"">",SG,UTA@NO,NO,39.82,7965,8300,-$335,42.45,7959,39.55,43.56,50.07,15.0,,,0.000000,0.000000
5,Rudy Gobert,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/goberru01.jpg"" width=""60"">",C,UTA@NO,UTA,34.02,6804,8200,-$1396,34.52,6472,40.11,34.24,30.37,14.0,,,0.000000,0.000000
6,Lonzo Ball,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/balllo01.jpg"" width=""60"">",PG,UTA@NO,NO,35.75,7150,7900,-$750,38.71,7258,32.61,40.48,43.33,15.0,,,0.000000,0.000000
7,Paul George\n🔥,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/georgpa01.jpg"" width=""60"">",SG,LAC@LAL,LAC,40.72,8144,7400,$744,42.44,7958,36.36,29.59,36.90,7.0,,,0.190433,0.000000
8,Brandon Ingram,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/ingrabr01.jpg"" width=""60"">",SF,UTA@NO,NO,31.69,6337,7300,-$963,33.71,6322,38.81,34.81,35.13,13.0,GTD,Face,0.000000,0.000000
9,Donovan Mitchell\n❄️❄️,"<img src=""https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/mitchdo01.jpg"" width=""60"">",SG,UTA@NO,UTA,38.43,7686,7200,$486,38.90,7295,36.10,34.89,25.80,19.0,,,0.000000,1.150799
10,Zion Williamson,"<img src=""https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg"" width=""60"">",PF,UTA@NO,NO,22.51,4501,7100,-$2599,26.14,4902,33.49,38.00,32.27,13.0,O,Personal,0.000000,0.000000


In [10]:
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(fd, 'fd.html', highlight_fd)
gen_page(dk, 'dk.html', highlight_fd)
gen_page(st, 'st.html', None)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

,Name,Picture,Position,Game,Team,Minutes,2PT FG,3PT FG,FTM,Rebounds,Assists,Blocks,Steals,Turnovers,Opponent Defensive Rank vs Position,Injury Indicator,Injury Details
1,LeBron James,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/jamesle01.jpg,SF,LAC@LAL,LAL,43.74,9.23,2.72,7.18,10.79,12.61,0.96,2.97,3.60,7.0,,
2,Anthony Davis,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/davisan02.jpg,PF,LAC@LAL,LAL,36.53,11.73,0.70,5.71,9.30,2.65,1.65,2.01,1.86,14.0,GTD,Eye
3,Kawhi Leonard,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/leonaka01.jpg,SF,LAC@LAL,LAC,35.54,8.54,2.14,7.84,7.23,3.49,0.78,1.95,2.48,7.0,,
4,Jrue Holiday,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/holidjr01.jpg,SG,UTA@NO,NO,27.51,3.94,2.69,3.82,4.77,7.64,0.00,1.95,2.98,15.0,,
5,Rudy Gobert,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/goberru01.jpg,C,UTA@NO,UTA,30.23,4.94,0.11,2.92,10.87,1.19,1.96,0.90,2.52,14.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Kostas Antetokounmpo,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/antetko01.jpg,PF,LAC@LAL,LAL,20.19,0.00,1.40,1.08,2.26,0.29,0.00,0.00,0.00,14.0,,
61,Landry Shamet,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/shamela01.jpg,PG,LAC@LAL,LAC,25.18,2.34,1.47,1.77,3.31,1.13,0.09,0.56,0.13,5.0,GTD,Illness
62,Miye Oni,https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg,SG,UTA@NO,UTA,16.33,0.00,0.61,0.00,1.58,0.00,0.00,0.00,0.00,19.0,,
63,Justin Wright-Foreman,https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg,PG,UTA@NO,UTA,15.88,0.00,0.65,0.00,1.22,0.00,0.00,0.00,0.00,26.0,,
